# WES237A Final Project


In [ ]:
import socket
import time
import string
import random
from pynq.overlays.base import BaseOverlay
from pynq.lib import Button

# Initialize PYNQ Board
base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODA
#include "gpio.h"

// Function to set a GPIO pin as output and write a value
void write_gpio(unsigned int pin, unsigned int val) {
    if (val > 1) {
        return;
    }
    gpio pin_out = gpio_open(pin); // Open the GPIO pin
    gpio_set_direction(pin_out, GPIO_OUT); // Set direction as output
    gpio_write(pin_out, val); // Write the specified value
}

// Function to generate a tone (buzzer) using GPIO
void tone(unsigned int pin, unsigned int frequency, float duration) {
    if (frequency == 0) {
        return;
    }
    unsigned int delay = 1000000 / frequency / 2; // Half-period delay
    unsigned int cycles = duration * frequency;
    
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);

    for (unsigned int i = 0; i < cycles; i++) {
        gpio_write(pin_out, 1);
        usleep(delay);
        gpio_write(pin_out, 0);
        usleep(delay);
    }
}

# Player 1: Server

In [ ]:
### Server Configuration
server_ip = "192.168.2.99"
listening_port = 8080
print(f"Player 1 (Server) starting on {server_ip}:{listening_port}")

# Setup Server Connection
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind((server_ip, listening_port))
sock.listen(1)
print(f"Server is waiting for Player 2 to connect on port {listening_port}...")
connection_socket, client_address = sock.accept()
print(f"Player 2 connected from {client_address}")

leds = base.leds  # LEDs for ship status indicator 
btns = base.buttons  # Buttons for user attack input
CONFIRM_BUTTON = 3  # Designate button 3 as the "RED" button


GRID_SIZE = 5   # Sets grid size for the game
total_ships = 1 # Total number of ships to in play

# Board 
player_grid = [["~"] * GRID_SIZE for _ in range(GRID_SIZE)]  # Player 2's hidden board
opponent_hidden_grid = [["~"] * GRID_SIZE for _ in range(GRID_SIZE)]  # Tracks hits/misses for Player 2's grid
opponent_ship_grid = [["~"] * GRID_SIZE for _ in range(GRID_SIZE)]  # Player 2's hidden ships
ship_status = [True] * total_ships  # Tracks Player 1's ship statuses

# Function to Play Sound for Hit/Miss Events also uses external buzzer from loT kit
def play_tone(pin, frequency, duration):  # pin 0 on PMODA 
    tone(pin, frequency, duration)

# Function to Play Victory Song with various of frequencies and same duration 
def play_victory_song():
    melody = [
        (1000, 0.3),
        (1200, 0.3),
        (1500, 0.3),
        (1800, 0.3),
    ]
    for freq, dur in melody:
        play_tone(0, freq, dur)
        time.sleep(0.1)

# Function to Update LED Status to show player visual feedback
def update_leds():
    for i in range(total_ships):
        if ship_status[i]:  # Ship active turn on LED
            leds[i].on()
        else:  # Ship sunk turn off LED 
            leds[i].off()

# Function to Display Two Boards
def display_boards():
    print("\n--- Your Board with Player 2's Attempts ---")
    print("   " + "   ".join(str(i) for i in range(GRID_SIZE)))
    for i, row in enumerate(player_grid):
        print(f"{string.ascii_uppercase[i]} | " + " | ".join(row) + " |")

    print("\n--- Opponent's Hidden Grid with Your Attempts ---")
    print("   " + "   ".join(str(i) for i in range(GRID_SIZE)))
    for i, row in enumerate(opponent_hidden_grid):
        print(f"{string.ascii_uppercase[i]} | " + " | ".join(row) + " |")
    print("\nLegend: S = Ship, X = Hit, O = Miss, ~ = Empty\n")

# Function to Place Ships
def place_ships(grid):
    print("\n--- Placement Phase: Place Your Ships ---")
    ships_left = total_ships
    while ships_left > 0:
        display_boards()
        print(f"Ships Remaining to Place: {ships_left}")

        row_input = input(f"Enter row (A to {string.ascii_uppercase[GRID_SIZE - 1]}): ").strip().upper()
        col_input = input(f"Enter column (0 to {GRID_SIZE - 1}): ").strip()

        try:
            row = string.ascii_uppercase.index(row_input)
            col = int(col_input)

            if grid[row][col] == "~":
                grid[row][col] = "S"
                leds[total_ships - ships_left].on()
                print(f"Ship placed at {row_input}{col}!")
                ships_left -= 1
            else:
                print("That spot is already occupied! Try again.")
        except (ValueError, IndexError):
            print("Invalid coordinates! Please try again.")

# Randomly Place Opponent Ships
def place_opponent_ships(grid):
    for _ in range(total_ships):
        while True:
            r, c = random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1)
            if grid[r][c] == "~":
                grid[r][c] = "S"
                break



# Synchronize Ship Placement
def sync_placement():
    print("Waiting for Player 2 to finish placing ships...")
    connection_socket.sendall("READY".encode())  # Notify Player 2 that Player 1 is ready
    response = connection_socket.recv(2048).decode()  # Wait for Player 2 to confirm readiness
    print("recv")
    if response == "READY":
        print("Both players are ready! Starting the game...")

# Check for Victory
def printgrid(grid):
    for i in range(5):
        for j in range(5):
            print(grid[i][j])
        print("\n")
        
def check_victory(grid):
    #printgrid(grid)
    for i in range(5):
        for j in range(5):
            if grid[i][j] == "S":
                return False
    return True
            
endgame = False            
# Player 1's Turn to Attack
def player1_attack():
    global endgame
    display_boards()
    row_input = input(f"Enter row (A to {string.ascii_uppercase[GRID_SIZE - 1]}): ").strip().upper()
    col_input = input(f"Enter column (0 to {GRID_SIZE - 1}): ").strip()

    try:
        row = string.ascii_uppercase.index(row_input)
        col = int(col_input)

        print(f"Press Button 3 to confirm attack!")
        while btns[CONFIRM_BUTTON].read() == 0:
            time.sleep(0.1)

        # Send attack to Player 2
        attack_message = f"{row},{col}"
        connection_socket.sendall(attack_message.encode())
        print("attack sent")
        # Process attack result
        result = connection_socket.recv(2048).decode()
        print("recv")
        if result == "HIT":
            print("HIT!")
            opponent_hidden_grid[row][col] = "X"
            play_tone(0, 1000, 0.3)
        elif result == "MISS":
            print("MISS!")
            opponent_hidden_grid[row][col] = "O"
            play_tone(0, 500, 0.3)
        elif result == "WIN":
            print("You won! All opponent ships have been sunk!")
            play_victory_song()
            endgame = True
            #exit()
    except (ValueError, IndexError):
        print("Invalid coordinates! Please try again.")

# Player 2's Turn
def player2_turn():
    global endgame
    print("Processing players2 turn")
    message = connection_socket.recv(2048).decode()
    print("recv")
    if message in ["HIT", "MISS", "WIN"]:
        if message == "HIT":
            print("Player 2 hit one of your ships!")
        elif message == "MISS":
            print("Player 2 missed!")
        elif message == "WIN":
            print("Player 2 won! All your ships have been sunk!")
            play_victory_song()
            #exit()
            endgame = True  
        return

    # Process attack from Player 2
    try:
        row, col = map(int, message.split(","))
        print(f"Player 2 attacked {string.ascii_uppercase[row]}{col}")
        if player_grid[row][col] == "S":
            player_grid[row][col] = "X"
            print("HIT!")
            play_tone(0, 1000, 0.3)
            update_leds()
            status = check_victory(player_grid)
            if status == True:
                connection_socket.sendall("WIN".encode())
                endgame = True  
                #exit()
            else: 
                connection_socket.sendall("HIT".encode())
        else:
            player_grid[row][col] = "O"
            print("MISS!")
            play_tone(0, 500, 0.3)
            connection_socket.sendall("MISS".encode())
            
    except (ValueError, IndexError):
        print("Invalid attack coordinates received. Ignoring...")

# Main Game Loop
def play_game():
    player1_starts = False
    #print("Player 1 starts!" if player1_starts else "Player 2 starts!")

    # Start the turn-based gameplay
    while endgame == False:
        if player1_starts:
            if endgame == False:
                print("\nPlayer 1's Turn")
                player1_attack()  # Player 1 takes their turn to attack
            if endgame == False:
                print("\nPlayer 2's Turn")
                player2_turn()  # Player 2 takes their turn to attack
        else:
            if endgame == False:
                print("\nPlayer 2's Turn")
                player2_turn()  # Player 2 takes their turn to attack
            if endgame == False:
                print("\nPlayer 1's Turn")
                player1_attack()  # Player 1 takes their turn to attack
    print("Game Over")
# Run the Game Setup
place_ships(player_grid)  # Player places ships
place_opponent_ships(opponent_ship_grid)  # Randomly place opponent's ships
sync_placement()  # Synchronize ship placement with Player 2
play_game()  # Start the game

# Client


In [ ]:
# Server Configuration
server_ip = "192.168.2.99"  # Replace with Players IP
listening_port = 8080 # Both players will have the same port number
print(f"Player 2 (Client) attempting to connect to {server_ip}:{listening_port}")


# Connect to Server
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((server_ip, listening_port))
print(f"Connected to Player 1 (Server) on port {listening_port}")

btns = base.buttons  # Buttons for user input
CONFIRM_BUTTON = 3  # Button 3 for confirmation

# Game Constants
GRID_SIZE = 5
total_ships = 4
player_grid = [["~"] * GRID_SIZE for _ in range(GRID_SIZE)]  # Player 2's board
opponent_hidden_grid = [["~"] * GRID_SIZE for _ in range(GRID_SIZE)]  # Tracks hits/misses for Player 1's grid
opponent_ship_grid = [["~"] * GRID_SIZE for _ in range(GRID_SIZE)]  # Player 1's hidden ships
ship_status = [True] * total_ships  # Tracks Player 2's ship statuses

# Function to Play Sound for Hit/Miss Events
def play_tone(pin, frequency, duration):
    """
    Plays a tone for the specified frequency and duration.
    """
    tone(pin, frequency, duration)

# Function to Play Victory Song
def play_victory_song():
    """
    Plays a short celebratory tune when a player wins.
    """
    melody = [
        (1000, 0.3),  # Frequency, Duration
        (1200, 0.3),
        (1500, 0.3),
        (1800, 0.3),
    ]
    for freq, dur in melody:
        play_tone(0, freq, dur)
        time.sleep(0.1)

# Function to Update LED Status
def update_leds():
    for i in range(total_ships):
        if ship_status[i]:  # Ship active
            base.leds[i].on()
        else:  # Ship sunk
            base.leds[i].off()

# Function to Display Two Boards (Player's Board and Opponent's Hidden Grid)
def display_boards():
    print("\n--- Your Board with Player 1's Attempts ---")
    print("   " + "   ".join(str(i) for i in range(GRID_SIZE)))
    for i, row in enumerate(player_grid):
        print(f"{string.ascii_uppercase[i]} | " + " | ".join(row) + " |")

    print("\n--- Opponent's Hidden Grid with Your Attempts ---")
    print("   " + "   ".join(str(i) for i in range(GRID_SIZE)))
    for i, row in enumerate(opponent_hidden_grid):
        print(f"{string.ascii_uppercase[i]} | " + " | ".join(row) + " |")
    print("\nLegend: S = Ship, X = Hit, O = Miss, ~ = Empty\n")

# Function to Place Ships
def place_ships(grid):
    print("\n--- Placement Phase: Place Your Ships ---")
    ships_left = total_ships
    while ships_left > 0:
        display_boards()
        print(f"Ships Remaining to Place: {ships_left}")

        row_input = input(f"Enter row (A to {string.ascii_uppercase[GRID_SIZE - 1]}): ").strip().upper()
        col_input = input(f"Enter column (0 to {GRID_SIZE - 1}): ").strip()

        try:
            row = string.ascii_uppercase.index(row_input)
            col = int(col_input)

            if grid[row][col] == "~":
                grid[row][col] = "S"
                base.leds[total_ships - ships_left].on()
                print(f"Ship placed at {row_input}{col}!")
                ships_left -= 1
            else:
                print("That spot is already occupied! Try again.")
        except (ValueError, IndexError):
            print("Invalid coordinates! Please try again.")

# Function to Synchronize Ship Placement
def sync_placement():
    response = sock.recv(2048).decode()  # Wait for Player 1 to signal readiness
    print("recv")
    if response == "READY":
        print("Player 1 is ready. Notifying Player 1...")
        sock.sendall("READY".encode())  # Notify Player 1 of readiness
        print("Both players are ready! Starting the game...")

def printgrid(grid):
    for i in range(5):
        for j in range(5):
            print(grid[i][j])
        print("\n")

# Check for Victory
def check_victory(grid):
    #printgrid(grid)
    for i in range(5):
        for j in range(5):
            if grid[i][j] == "S" :
                return False
    return True
# Function for Player 2 to Take a Turn and Attack
endgame = False

def player2_attack():
    global endgame
    display_boards()
    row_input = input(f"Enter row (A to {string.ascii_uppercase[GRID_SIZE - 1]}): ").strip().upper()
    col_input = input(f"Enter column (0 to {GRID_SIZE - 1}): ").strip()

    try:
        row = string.ascii_uppercase.index(row_input)
        col = int(col_input)

        print(f"Press Button 3 to confirm attack!")
        while btns[CONFIRM_BUTTON].read() == 0:
            time.sleep(0.1)

        # Send attack to Player 1
        attack_message = f"{row},{col}"
        sock.sendall(attack_message.encode())
        print("attack sent")
        # Receive result of attack
        result = sock.recv(2048).decode()
        if result == "HIT":
            print("HIT!")
            opponent_hidden_grid[row][col] = "X"
            play_tone(0, 1000, 0.3)  # Hit sound
        elif result == "MISS":
            print("MISS!")
            opponent_hidden_grid[row][col] = "O"
            play_tone(0, 500, 0.3)  # Miss sound
        elif result == "WIN":
            print("You won! All opponent ships have been sunk!")
            #sock.sendall("WIN".encode())
            play_victory_song()  # Play victory tune
            #exit()  # End the game
            endgame = True

    except (ValueError, IndexError):
        print("Invalid coordinates! Please try again.")

# Function for Player 1's Turn
def player1_turn():
    global endgame
    # Receive attack from Player 1
    message = sock.recv(2048).decode()
    row, col = map(int, message.split(","))
    print(f"Player 1 attacked {string.ascii_uppercase[row]}{col}")

    if player_grid[row][col] == "S":
        player_grid[row][col] = "X"
        print("HIT!")
        play_tone(0, 1000, 0.3)
        ship_status[total_ships - 1] = False
        update_leds()
        status = check_victory(player_grid)
        if status == True:
            sock.sendall("WIN".encode())
            #exit()
            endgame = True
        else: 
            sock.sendall("HIT".encode())
    else:
        print("MISS!")
        play_tone(0, 500, 0.3)
        sock.sendall("MISS".encode())

     # Main Game Loop
def play_game():
    player2_starts = True
    #player2_starts = random.choice([True, False])  # Randomly decide who starts
    print("Player 2 starts!" if player2_starts else "Player 1 starts!")
    
    # Start the turn-based gameplay
    while endgame == False:
        if player2_starts:
            print("\nPlayer 2's Turn")
            if endgame == False:
                player2_attack()  # Player 2 takes their turn to attack

            # Check if Player 1 has lost
            print("\nPlayer 1's Turn")
            if endgame == False:
                player1_turn()  # Player 1 takes their turn to attack

        else:
            print("\nPlayer 1's Turn")
            if endgame == False:
                player1_turn()  # Player 1 takes their turn to attack
            # Check if Player 2 has lost
            print("\nPlayer 2's Turn")
            if endgame == False:
                player2_attack()  # Player 2 takes their turn to attack
    print("Game Over")


# Setup Phase
print("\nPlayer 2: Place Your Ships")
place_ships(player_grid)
sync_placement()  # Wait for both players to finish placing ships
play_game()  # Start Gameplay
sock.close()